# SMT

## Import and utils

In [51]:
from z3 import *

## Read values from the file

In [52]:
# Read the file and initialize the variables
with open("../data/inst02.dat", "r") as file:
    lines = file.readlines()

In [53]:
# CREATE SOLVER INSTANCE
#solver = Solver()
solver = Optimize()

## Decision variables

In [54]:
# Parse the values from the file
m = int(lines[0].strip())  # Number of couriers
n = int(lines[1].strip())  # Number of items
l_values = list(map(int, lines[2].split()))  # Maximum load for each courier
l = Array('l', IntSort(), IntSort())
for i in range(m):
    solver.add(l[i] == l_values[i])

s_values = list(map(int, lines[3].split()))  # Size of each item
s = Array('s', IntSort(), IntSort())

for i in range(n):
    solver.add(s[i] == s_values[i])

D_values = [list(map(int, line.split())) for line in lines[4:]]  # Distance matrix
D = Array('D', IntSort(), ArraySort(IntSort(), IntSort()))  # Routes for each courier

# Set the matrix of distances D to the default values
for i in range(n+1):
#    #a_i = Array('a_{i}', IntSort(), IntSort())
    for j in range(n+1):
        solver.add(D[i][j] == D_values[i][j])

routes = Array('routes', IntSort(), ArraySort(IntSort(), IntSort()))  # Routes for each courier

#D = [[Int(f"D_{i}_{j}") for j in range(n + 1)] for i in range(n + 1)]
#for i in range(n + 1):
#    for j in range(n + 1):
#        D[i][j] = D_values[i][j]
#print(s)

In [55]:
#Useful functions
def z3_max(vector):
    maximum = vector[0]
    for value in vector[1:]:
        maximum = If(value > maximum, value, maximum)
    return maximum

def z3_min(vector):
    minimum = vector[0]
    for i in range(1,n):
        value = vector[i]
        minimum = If(value < minimum, value, minimum)
    return minimum

In [56]:
# DOMAIN CONSTRAINTS
for i in range(m):
    for t in range(n):
        solver.add(And(routes[i][t] >= 1, routes[i][t] <= n + 1))

## Constraints

In [57]:
# TODO: FIX CONSTRAINTS
# All the values from 1 to n need to appear exactly once in the routes matrix
for p in range(1,n+1):
   solver.add(Sum([If(routes[i][j] == p, 1, 0) for i in range(m) for j in range(n+1)]) == 1)


for i in range(m):
    #Force the value n+1 to appear at least once.
    #In the worst case scenario a courier will deliver n-m+1 items, 
    # each courier will have a n+1 in the positions n-m+1..n because we forced each courier to deliver at least one item.
    for j in range(n-m+1, n+1):
        solver.add(routes[i][j] == n+1)
    
    #Constraint to force that the total size of items assigned to each courier cannot exceed their maximum load size and constraint to force that the total size of items assigned to each courier is at least the load of the min value of item sizes
    sums = Sum([If(routes[i][j] < n+1, s[routes[i][j] -1], 0) for j in range(n+1)])
    solver.add(sums <= l[i])
    
    
    #Simmetry breaking
    #Constraint to force the first value of each row of the matrix to be different from n+1
    solver.add(routes[i][0] < n+1)

    #Constraint to force all the numbers after the first n+1 to be also n+1
    for j in range(n):
        solver.add(If(routes[i][j] == n+1, routes[i][j+1] == n+1, True))
    
     
##IMPLICIT CONSTRAINT
## Constraint to force exactly (m * (n + 1)) - n n+1s
solver.add(Sum([If(routes[i][j] == n + 1, 1, 0) for i in range(m) for j in range(n+1)]) == (m * (n + 1)) - n)

In [58]:
# OBJECTIVE FUNCTION
# Application of the same objective function above to all couriers
#dist_courier = [(Sum( [D[routes[i][j]][routes[i][j + 1]] for j in range(n)] )+ D[n + 1][routes[i][0]] )  for i in range(m)]
dist_courier = [(Sum( [D[routes[i][j]-1][routes[i][j + 1]-1] for j in range(n)] )+ D[n][routes[i][0]-1] )  for i in range(m)]


maximum = z3_max(dist_courier)



In [59]:
solver.minimize(maximum)
if solver.check() == sat:
    model = solver.model()
    length = model.evaluate(maximum)
    
    
    routes_sol = []
    for i in range(m):
        routes_sol.append([model.evaluate(routes[i][j]).as_long() for j in range(n+1)])
    print(routes_sol)
    print(length)
    print([model.evaluate(v) for v in dist_courier])
else:
    print("no sat")
#TODO: Create function that prints the results


[[5, 10, 10, 10, 10, 10, 10, 10, 10, 10], [9, 10, 10, 10, 10, 10, 10, 10, 10, 10], [8, 10, 10, 10, 10, 10, 10, 10, 10, 10], [4, 1, 10, 10, 10, 10, 10, 10, 10, 10], [6, 10, 10, 10, 10, 10, 10, 10, 10, 10], [2, 7, 3, 10, 10, 10, 10, 10, 10, 10]]
226
[192, 132, 174, 209, 192, 226]


In [60]:
print('---n---')
print(n)
print("---SIZES---")
print([model.evaluate(s[i]) for i in range(n) ])
print("---LOADS---")
print([model.evaluate(l[i]) for i in range(m) ])
print('---DISTANCES---')  
for i in range(n+1):
    row = []
    for j in range(n+1):
        a = model.evaluate(D[i][j]).as_long()
        row.append(a)
    print(row)

print('---ROUTES---')  
for i in range(m):
    row = []
    for j in range(n+1):
        a = model.evaluate(routes[i][j]).as_long()
        row.append(a)
    print(row)

print('---DIST COURIER---')  
print([model.evaluate(dist_courier[i]) for i in range(m)])




---n---
9
---SIZES---
[11, 11, 23, 16, 2, 1, 24, 14, 20]
---LOADS---
[190, 185, 185, 190, 195, 185]
---DISTANCES---
[0, 199, 119, 28, 179, 77, 145, 61, 123, 87]
[199, 0, 81, 206, 38, 122, 55, 138, 76, 113]
[119, 81, 0, 126, 69, 121, 26, 117, 91, 32]
[28, 206, 126, 0, 186, 84, 152, 68, 130, 94]
[169, 38, 79, 176, 0, 92, 58, 108, 46, 98]
[77, 122, 121, 84, 102, 0, 100, 16, 46, 96]
[145, 55, 26, 152, 58, 100, 0, 91, 70, 58]
[61, 138, 113, 68, 118, 16, 91, 0, 62, 87]
[123, 76, 91, 130, 56, 46, 70, 62, 0, 66]
[87, 113, 32, 94, 94, 96, 58, 87, 66, 0]
---ROUTES---
[5, 10, 10, 10, 10, 10, 10, 10, 10, 10]
[9, 10, 10, 10, 10, 10, 10, 10, 10, 10]
[8, 10, 10, 10, 10, 10, 10, 10, 10, 10]
[4, 1, 10, 10, 10, 10, 10, 10, 10, 10]
[6, 10, 10, 10, 10, 10, 10, 10, 10, 10]
[2, 7, 3, 10, 10, 10, 10, 10, 10, 10]
---DIST COURIER---
[192, 132, 174, 209, 192, 226]
